PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=fc110f45c0ac8bc28b29a23da52342b1450a85495033299da5b6e94c68e201b8
  Stored in directory: /root/.cache/pip/wheels/97/f5/c0/947e2c0942b361ffe58651f36bd7f13772675b3863fd63d1b1
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.4
    Uninstalling pyspark-3.5.4:
      Successfully uninstalled pyspark-3.5.4


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Taipei Housing Price Prediction") \
    .getOrCreate()

# 타이베이 주택 가격 예측 모델 만들기




데이터셋 설명

이번 문제는 대만 타이베이 시의 신단 지역에서 수집된 주택 거래 관련 정보를 바탕으로 주택 가격(정확히는 주택의 평당 가격)을 예측하는 Regression 모델을 만들어보는 것이다. 총 6개의 피쳐와 주택의 평당 가격에 해당하는 레이블 정보가 훈련 데이터로 제공된다. 레이블의 경우에는 주택의 최종 가격이 아니라 평당 가격이란 점을 다시 한번 강조한다.

각 컬럼에 대한 설명은 아래와 같으며 모든 필드는 X4를 제외하고는 실수 타입이다.

* X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 2013년 3월임을 나타낸다 (0.250 = 3/12)
* X2: 주택 나이 (년수)
* X3: 가장 가까운 지하철역까지의 거리 (미터)
* X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
* X5: 주택 위치의 위도 (latitude)
* X6: 주택 위치의 경도 (longitude)
* Y: 주택 평당 가격



In [3]:
spark

In [4]:
!wget https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv

--2025-02-24 09:02:14--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.204.42, 3.5.141.151, 3.5.184.161, ...
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.204.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20014 (20K) [text/csv]
Saving to: ‘Taipei_sindan_housing.csv’

Taipei_sindan_housi 100%[===================>]  19.54K  --.-KB/s    in 0s      

2025-02-24 09:02:15 (86.6 MB/s) - ‘Taipei_sindan_housing.csv’ saved [20014/20014]



In [5]:
!ls -tl

total 24
drwxr-xr-x 1 root root  4096 Feb 20 14:24 sample_data
-rw-r--r-- 1 root root 20014 Jul 17  2021 Taipei_sindan_housing.csv


In [6]:
data = spark.read.csv('./Taipei_sindan_housing.csv', header=True, inferSchema=True)

In [7]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: integer (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- Y: double (nullable = true)



In [8]:
data.show()

+--------+----+--------+---+--------+---------+----+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|
+--------+----+--------+---+--------+---------+----+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|
|2013.083|34.8|405.2134|  1|24.97349|121.53372|41.4|
|2013.333| 6.3|90.45606|  9|24.97433| 121.5431|58.1|
|2012.917|13.0|492.2313|  5|24.96515|121.53737|39.3|
|2012.667|20.4|2469.645|  4|24.96108|121.51046|23.8|
|  2013.5|13.2|1164.838|  4|24.99156|121.53406|34.3|
|2013.583|35.7|579.2083|  2| 24.9824|121.54619

In [9]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6']
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
data_transformed = assembler.transform(data)

In [10]:
data_transformed.show()

+--------+----+--------+---+--------+---------+----+--------------------+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|            features|
+--------+----+--------+---+--------+---------+----+--------------------+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|[2012.917,32.0,84...|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|[2012.917,19.5,30...|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|[2013.583,13.3,56...|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|[2013.5,13.3,561....|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|[2012.833,5.0,390...|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|[2012.667,7.1,217...|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|[2012.667,34.5,62...|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|[2013.417,20.3,28...|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|[2013.5,31.7,5512...|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|[2013.417,17.9,17...|
|2013.083|34.8|405.2134|  1|24.97349|1

In [11]:
train_data, test_data = data_transformed.randomSplit([0.8, 0.2], seed=42)

In [12]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='Y')
model = lr.fit(train_data)

In [13]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(test_data)
evaluator = RegressionEvaluator(labelCol='Y', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(predictions)

In [14]:
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 12.185242864996995


In [15]:
predictions.select('Y', 'prediction').show(10)

+----+------------------+
|   Y|        prediction|
+----+------------------+
|47.7| 47.66368763873288|
|50.0| 50.51857823951468|
|32.1|31.476266262681747|
|37.4|39.547818471113715|
|40.3| 37.05396283416485|
|39.7| 45.65478150515446|
|31.9| 28.03200203106462|
|38.4| 43.18159862007815|
|55.1|47.497004237857254|
|37.5| 37.26886713503882|
+----+------------------+
only showing top 10 rows

